# Import

In [1]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 98 kB 3.8 MB/s 


In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
import keras_tuner as kt

# Loading Data

In [3]:
(img_train, label_train), (img_test, label_test) = fashion_mnist.load_data()
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

4431872/4422102 [==============================] - 0s 0us/step


# Base Model

In [4]:
def model_builder():
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(10))

    model.compile(
        optimizer=Adam(learning_rate=0.01),
        loss=SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    return model

In [5]:
model = model_builder()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(
    img_train,
    label_train,
    epochs=50,
    validation_split=0.2
)

Epoch 1/50
1500/1500 [==============================] - 8s 4ms/step - loss: 0.5627 - accuracy: 0.8102 - val_loss: 0.4611 - val_accuracy: 0.8416
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4407 - accuracy: 0.8447 - val_loss: 0.4259 - val_accuracy: 0.8497
Epoch 3/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4129 - accuracy: 0.8550 - val_loss: 0.4063 - val_accuracy: 0.8577
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3903 - accuracy: 0.8587 - val_loss: 0.4151 - val_accuracy: 0.8565
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3827 - accuracy: 0.8645 - val_loss: 0.4176 - val_accuracy: 0.8566
Epoch 6/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3773 - accuracy: 0.8659 - val_loss: 0.4608 - val_accuracy: 0.8473
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3689 - accuracy: 0.8678 - val_loss: 0.4676 - val_accuracy:

In [7]:
model.evaluate(
    x=img_test,
    y=label_test,
)

313/313 [==============================] - 1s 3ms/step - loss: 0.7085 - accuracy: 0.8490


[0.7084887623786926, 0.8489999771118164]

# Keras Tuner

In [8]:
def model_builder(hp):
    hidden_units = hp.Int(
        name='hidden_units',
        min_value=16,
        max_value=512,
        step=16
    )
    learning_rate = hp.Choice(
        'learning_rate',
        values=[0.0001, 0.0005, 0.001, 0.005, 0.01]
    )

    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(units=hidden_units, activation='relu'))
    model.add(Dense(10))

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss=SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    return model

In [9]:
MAX_EPOCHS = 50
EPOCHS = 50
BATCH_SIZE = 32

In [10]:
tuner = kt.Hyperband(
    hypermodel=model_builder,
    objective='val_accuracy',
    max_epochs=MAX_EPOCHS,
    directory='./tuner_results',
)

In [11]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
hidden_units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 512, 'step': 16, 'sampling': None}
learning_rate (Choice)
{'default': 0.0001, 'conditions': [], 'values': [0.0001, 0.0005, 0.001, 0.005, 0.01], 'ordered': True}


In [12]:
tuner.search(
    img_train,
    label_train,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
)

Trial 90 Complete [00h 04m 10s]
val_accuracy: 0.8945833444595337

Best val_accuracy So Far: 0.9020000100135803
Total elapsed time: 01h 33m 28s
INFO:tensorflow:Oracle triggered exit


In [13]:
tuner.results_summary()

Results summary
Results in ./tuner_results/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
hidden_units: 432
learning_rate: 0.0005
tuner/epochs: 50
tuner/initial_epoch: 17
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: bb38582319ee5054c42e3dfa5603bdf5
Score: 0.9020000100135803
Trial summary
Hyperparameters:
hidden_units: 480
learning_rate: 0.0005
tuner/epochs: 50
tuner/initial_epoch: 17
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 0a4af1d3711c6cb23ae518ad3bc3204c
Score: 0.9009166955947876
Trial summary
Hyperparameters:
hidden_units: 352
learning_rate: 0.0005
tuner/epochs: 50
tuner/initial_epoch: 17
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: bc64df6525ad7a143c46956d97bfa979
Score: 0.9009166955947876
Trial summary
Hyperparameters:
hidden_units: 368
learning_rate: 0.0005
tuner/epochs: 50
tuner/initial_epoch: 17
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 98c3ec15511fe6b97bac6fe69964825a
Score: 0.8995000123

In [20]:
best_hp = tuner.get_best_hyperparameters(2)[0]
best_model = model_builder(best_hp)
best_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 432)               339120    
                                                                 
 dense_5 (Dense)             (None, 10)                4330      
                                                                 
Total params: 343,450
Trainable params: 343,450
Non-trainable params: 0
_________________________________________________________________


In [21]:
checkpoint = ModelCheckpoint(
    filepath='./checkpoints/evaluater.hdf5',
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
)

best_model.fit(
    x=img_train,
    y=label_train,
    validation_split=0.2,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint]
)

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5089 - accuracy: 0.8214 - val_loss: 0.4246 - val_accuracy: 0.8443
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3785 - accuracy: 0.8640 - val_loss: 0.3816 - val_accuracy: 0.8590
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3370 - accuracy: 0.8777 - val_loss: 0.3494 - val_accuracy: 0.8737
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3089 - accuracy: 0.8869 - val_loss: 0.3298 - val_accuracy: 0.8784
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2878 - accuracy: 0.8936 - val_loss: 0.3222 - val_accuracy: 0.8813
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2716 - accuracy: 0.8998 - val_loss: 0.3237 - val_accuracy: 0.8813
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2598 - accuracy: 0.9041 - val_loss: 0.3154 - val_accuracy:

In [25]:
evaluater = load_model('./checkpoints/evaluater.hdf5')

In [26]:
evaluater.evaluate(
    x=img_train,
    y=label_train,
    batch_size=BATCH_SIZE,
)

1875/1875 [==============================] - 4s 2ms/step - loss: 0.1588 - accuracy: 0.9456


[0.15877357125282288, 0.9456166625022888]

In [27]:
evaluater.evaluate(
    x=img_test,
    y=label_test,
    batch_size=BATCH_SIZE,
)

313/313 [==============================] - 1s 3ms/step - loss: 0.3823 - accuracy: 0.8907


[0.38228559494018555, 0.8906999826431274]